In [83]:
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
from tqdm.contrib import tzip
import numpy as np

WD = 'C:\\Users\Sunwoo\Desktop\업무\code\[코드스테이츠] AIB16\AI_16_허선우_Section2_Project\Data'
os.chdir(WD)

In [54]:
def make_dataset(wd):
    
    wd_list = sorted(glob(wd))
    df = pd.DataFrame()
    
    for fn in tqdm(wd_list):
        
        df_new = pd.read_excel(fn, index_col = 0)
        df = pd.concat([df, df_new])
        
    return df

dataset_financial = make_dataset(wd = "수집/*.xlsx")
dataset_financial.to_csv('dataset_finacial.csv')

100%|██████████| 36/36 [00:33<00:00,  1.06it/s]


In [71]:
dataset_financial = pd.read_csv('dataset_finacial.csv', index_col = 0).iloc[:,:-9]
dataset_financial['결산년도'] = pd.to_datetime(dataset_financial['결산년도'])
dataset_financial = dataset_financial.drop_duplicates()
dataset_financial

,결산년도,총자산,자본금,자본총계,매출액,영업이익,당기순이익,사업자등록번호
0,2020-12-31,7862847.0,1410020.0,7303383.0,2535150.0,-3031171.0,-3367400.0,221-81-48293
1,2019-12-31,11322266.0,1410020.0,10670783.0,2311466.0,-26805.0,278091.0,221-81-48293
2,2018-12-31,10791503.0,1410020.0,10392692.0,3042851.0,445686.0,400916.0,221-81-48293
3,2017-12-31,10434592.0,1410020.0,9991776.0,2888489.0,121397.0,-1226084.0,221-81-48293
4,2016-12-31,11630780.0,1410020.0,11217860.0,2246411.0,-115412.0,164513.0,221-81-48293
...,...,...,...,...,...,...,...,...
0,2020-12-31,12979803.0,150000.0,5319592.0,75626777.0,-445496.0,-400806.0,315-81-32820
1,2019-12-31,10142990.0,150000.0,5720398.0,83810901.0,-198279.0,-243281.0,315-81-32820
2,2018-12-31,11527759.0,150000.0,7763679.0,71428798.0,594055.0,387409.0,315-81-32820
3,2017-12-31,11928022.0,150000.0,7376269.0,62967410.0,428346.0,302429.0,315-81-32820


In [72]:
dataset_financial_new = dataset_financial.groupby(['사업자등록번호', '결산년도']).mean().reset_index()
dataset_financial_new['year'] = pd.to_datetime(dataset_financial_new['결산년도']).dt.year
dataset_financial_new

,사업자등록번호,결산년도,총자산,자본금,자본총계,매출액,영업이익,당기순이익,year
0,101-81-00452,2016-12-31,301385318.0,49777345.0,-59131568.0,369099317.0,-18544133.0,-13342241.0,2016
1,101-81-00452,2017-12-31,178830491.0,49777345.0,-105731559.0,255915468.0,-24057704.0,-48840178.0,2017
2,101-81-00452,2018-12-31,54033178.0,13000800.0,7412704.0,87056333.0,2539046.0,-4207220.0,2018
3,101-81-00452,2019-12-31,195944350.0,31500000.0,44858662.0,3053778.0,-8973868.0,-10866226.0,2019
4,101-81-00452,2020-12-31,237408673.0,31500000.0,61805343.0,305747696.0,18904750.0,16832350.0,2020
...,...,...,...,...,...,...,...,...,...
319500,899-88-01033,2019-12-31,116286.0,10000.0,62824.0,1851283.0,48284.0,48284.0,2019
319501,899-88-01033,2020-12-31,266969.0,100000.0,238968.0,1261365.0,86119.0,86143.0,2020
319502,899-88-01203,2018-12-31,86398.0,10000.0,13195.0,60250.0,3546.0,3195.0,2018
319503,899-88-01203,2019-12-31,449408.0,10000.0,206300.0,3919729.0,208195.0,193104.0,2019


In [73]:
bsno_dataset = pd.read_csv('bsno_dataset.csv', index_col = 0)
bsno_dataset['폐업일자'] = bsno_dataset['사업자등록상태'].str.split('폐업일자:').str[1].str.extract(r'(\d+-\d+-\d+)')
bsno_dataset['폐업일자'] = pd.to_datetime(bsno_dataset['폐업일자'])
bsno_dataset['폐업년도'] = bsno_dataset['폐업일자'].dt.year
bsno_dataset

,사업자등록번호,사업자등록상태,조회기준일자,폐업일자,폐업년도
0,101-81-00452,부가가치세 일반과세자 입니다.,2022-12-04,NaT,NaN
0,101-81-24696,부가가치세 일반과세자 입니다.,2022-12-04,NaT,NaN
0,101-81-62477,부가가치세 일반과세자 입니다.,2022-12-04,NaT,NaN
0,105-81-68075,부가가치세 일반과세자 입니다.,2022-12-04,NaT,NaN
0,128-86-79624,부가가치세 일반과세자 입니다.,2022-12-04,NaT,NaN
...,...,...,...,...,...
0,767-81-01284,부가가치세 일반과세자 입니다.,2022-12-07,NaT,NaN
0,873-86-01911,부가가치세 일반과세자 입니다.,2022-12-07,NaT,NaN
0,108-81-83610,부가가치세 일반과세자 입니다.,2022-12-07,NaT,NaN
0,120-81-01336,부가가치세 일반과세자 입니다.,2022-12-07,NaT,NaN


In [74]:
fi_data = dataset_financial_new.drop_duplicates(['사업자등록번호', 'year'], keep = 'last')
fi_data = fi_data.drop(columns = ['결산년도'])
fi_data = fi_data.query( '2015 <= year <= 2021')
fi_data

,사업자등록번호,총자산,자본금,자본총계,매출액,영업이익,당기순이익,year
0,101-81-00452,301385318.0,49777345.0,-59131568.0,369099317.0,-18544133.0,-13342241.0,2016
1,101-81-00452,178830491.0,49777345.0,-105731559.0,255915468.0,-24057704.0,-48840178.0,2017
2,101-81-00452,54033178.0,13000800.0,7412704.0,87056333.0,2539046.0,-4207220.0,2018
3,101-81-00452,195944350.0,31500000.0,44858662.0,3053778.0,-8973868.0,-10866226.0,2019
4,101-81-00452,237408673.0,31500000.0,61805343.0,305747696.0,18904750.0,16832350.0,2020
...,...,...,...,...,...,...,...,...
319500,899-88-01033,116286.0,10000.0,62824.0,1851283.0,48284.0,48284.0,2019
319501,899-88-01033,266969.0,100000.0,238968.0,1261365.0,86119.0,86143.0,2020
319502,899-88-01203,86398.0,10000.0,13195.0,60250.0,3546.0,3195.0,2018
319503,899-88-01203,449408.0,10000.0,206300.0,3919729.0,208195.0,193104.0,2019


In [76]:
# feature_engineering

# 활동성 지표
fi_data['총자산회전율'] = fi_data['매출액'] / fi_data['총자산']


# 수익성 지표
fi_data['매출액순이익률'] = fi_data['당기순이익'] / fi_data['매출액']
fi_data['총자산순이익률'] = fi_data['당기순이익'] / fi_data['총자산']
fi_data['자기자본순이익률'] = fi_data['당기순이익'] / fi_data['자본금']


# 안정성
fi_data['총부채'] = fi_data['총자산'] - fi_data['자본총계'] 
fi_data['부채비율'] = fi_data['총부채'] / fi_data['총자산'] 
fi_data['자기자본비율'] = fi_data['자본금'] / fi_data['총자산'] 

fi_data_new = fi_data.groupby(['사업자등록번호', 'year']).mean().unstack()
fi_data_new

총자산                                                      \
year             2015         2016         2017         2018         2019   
사업자등록번호                                                                     
101-81-00452      NaN  301385318.0  178830491.0   54033178.0  195944350.0   
101-81-10173      NaN          NaN          NaN          NaN          NaN   
101-81-14655      NaN  502877296.0  508337472.0  496188155.0  511088533.0   
101-81-16987  69357.0      86522.0     181628.0          NaN          NaN   
101-81-18985      NaN          NaN          NaN          NaN    3549456.0   
...               ...          ...          ...          ...          ...   
899-88-00465      NaN      57727.0     647012.0     545642.0     766434.0   
899-88-00499      NaN          NaN          NaN     113393.0      96513.0   
899-88-00897      NaN          NaN          NaN     221506.0     485144.0   
899-88-01033      NaN          NaN          NaN      63355.0     116286.0   
899-88-01203      NaN          NaN          NaN      86398.0     449408.0   

                                          자본금                          ...  \
year                 2020      2021      2015        2016        2017  ...   
사업자등록번호                                                                ...   
101-81-00452  237408673.0       NaN       NaN  49777345.0  49777345.0  ...   
101-81-10173     236219.0  247054.0       NaN         NaN         NaN  ...   
101-81-14655  545268737.0       NaN       NaN   2739630.0   2739630.0  ...   
101-81-16987          NaN       NaN  150000.0    150000.0    150000.0  ...   
101-81-18985    2711347.0       NaN       NaN         NaN         NaN  ...   
...                   ...       ...       ...         ...         ...  ...   
899-88-00465     853001.0       NaN       NaN     50000.0     50000.0  ...   
899-88-00499      86448.0       NaN       NaN         NaN         NaN  ...   
899-88-00897          NaN       NaN       NaN         NaN         NaN  ...   
899-88-01033     266969.0       NaN       NaN         NaN         NaN  ...   
899-88-01203    1985871.0       NaN       NaN         NaN         NaN  ...   

                  부채비율                         자기자본비율                      \
year              2019       2020      2021      2015      2016      2017   
사업자등록번호                                                                     
101-81-00452  0.771064   0.739667       NaN       NaN  0.165162  0.278349   
101-81-10173       NaN   0.279990  0.247294       NaN       NaN       NaN   
101-81-14655  0.203961   0.255592       NaN       NaN  0.005448  0.005389   
101-81-16987       NaN        NaN       NaN  2.162723  1.733663  0.825864   
101-81-18985  7.714753  10.347100       NaN       NaN       NaN       NaN   
...                ...        ...       ...       ...       ...       ...   
899-88-00465  0.920114   0.888859       NaN       NaN  0.866146  0.077278   
899-88-00499  0.041901   0.025009       NaN       NaN       NaN       NaN   
899-88-00897  1.115630        NaN       NaN       NaN       NaN       NaN   
899-88-01033  0.459746   0.104885       NaN       NaN       NaN       NaN   
899-88-01203  0.540952   0.416039       NaN       NaN       NaN       NaN   

                                                      
year              2018      2019      2020      2021  
사업자등록번호                                               
101-81-00452  0.240608  0.160760  0.132683       NaN  
101-81-10173       NaN       NaN  0.656171  0.627393  
101-81-14655  0.005521  0.005360  0.005024       NaN  
101-81-16987       NaN       NaN       NaN       NaN  
101-81-18985       NaN  1.176794  1.540555       NaN  
...                ...       ...       ...       ...  
899-88-00465  0.091635  0.065237  0.058617       NaN  
899-88-00499  1.851966  2.175873  2.429206       NaN  
899-88-00897  0.316019  0.144287       NaN       NaN  
899-88-01033  0.157841  0.085995  0.374575       NaN  
899-88-01203  0.115743  0.022251  0.005036       NaN

In [77]:
#성장성지표

for i in [2015, 2016, 2017, 2018, 2019, 2020]:
    fi_data_new[('총자본증가율', i+1)] = (fi_data_new[('자본총계', i+1)]/fi_data_new[('자본총계', i)])-1
    fi_data_new[('총부채증가율', i+1)] = (fi_data_new[('총부채', i+1)]/fi_data_new[('총부채', i)])-1
    fi_data_new[('매출액증가율', i+1)] = (fi_data_new[('매출액', i+1)]/fi_data_new[('매출액', i)])-1
    fi_data_new[('영업이익증가율', i+1)] = (fi_data_new[('영업이익', i+1)]/fi_data_new[('영업이익', i)])-1
    fi_data_new[('순이익증가율', i+1)] = (fi_data_new[('당기순이익', i+1)]/fi_data_new[('당기순이익', i)])-1

fi_data_new[('총자본증가율')]

year,2016,2017,2018,2019,2020,2021
사업자등록번호,,,,,,
101-81-00452,NaN,0.788073,-1.070109,5.051592,0.377779,NaN
101-81-10173,NaN,NaN,NaN,NaN,NaN,0.093362
101-81-14655,NaN,-0.038857,-0.049352,0.009974,-0.002320,NaN
101-81-16987,-0.032907,0.048231,NaN,NaN,NaN,NaN
101-81-18985,NaN,NaN,NaN,NaN,0.063335,NaN
...,...,...,...,...,...,...
899-88-00465,NaN,2.091562,2.037297,-1.314057,0.548386,NaN
899-88-00499,NaN,NaN,NaN,-0.156128,-0.088495,NaN
899-88-00897,NaN,NaN,NaN,-1.399335,NaN,NaN


In [78]:
# target
fi_data_new[('y', 2018)] = (~fi_data_new['총자산'][2019].isnull()).astype('int')
fi_data_new[('y', 2019)] = (~fi_data_new['총자산'][2020].isnull()).astype('int')
fi_data_new[('y', 2020)] = (~fi_data_new['총자산'][2021].isnull()).astype('int')

In [79]:
train_query = '2018 == year'# >= 2016'
test_query = '2019 == year'# >= 2017'
test_query2 = '2020 == year'# >= 2018'
test_query3 = '2021 == year'# >= 2019'

info = pd.read_csv('infodata.csv', index_col = 0)

Xy_2018 = fi_data_new.stack().reset_index().query(train_query).groupby(['사업자등록번호', 'year']).mean().unstack()
Xy_2019 = fi_data_new.stack().reset_index().query(test_query).groupby(['사업자등록번호', 'year']).mean().unstack()
Xy_2020 = fi_data_new.stack().reset_index().query(test_query2).groupby(['사업자등록번호', 'year']).mean().unstack()
Xy_2021 = fi_data_new.stack().reset_index().query(test_query3).groupby(['사업자등록번호', 'year']).mean().unstack()
Xy_2021[('y', 2021)] = pd.concat([Xy_2021, info.set_index('사업자등록번호')],axis=1, join='inner')['업소상태']

In [80]:
from sklearn.model_selection import train_test_split
X_2018 = Xy_2018.drop(columns = ['y']).dropna()  
y_2018 = Xy_2018['y'][2018][X_2018.index]

X_2019 =  Xy_2019.drop(columns = ['y']).dropna()
y_2019 = Xy_2019['y'][2019][X_2019.index]

X_2020 =  Xy_2020.drop(columns = ['y']).dropna()
y_2020 = Xy_2020['y'][2020][X_2020.index]

X_2021 =  Xy_2021.drop(columns = ['y'])
y_2021 = Xy_2021['y'][2021][X_2021.index]

def make_X(X):
    info_drop = info.drop(columns =['업소상태'])
    X = pd.merge(X.reset_index(), info_drop, how ='left', on='사업자등록번호')
    X.drop(columns =  [('사업자등록번호', '')], inplace = True)
    X.replace([np.inf, -np.inf], np.nan, inplace = True)
    return X

In [82]:
make_X(X_2018).to_excel('X_2018.xlsx')
make_X(X_2019).to_excel('X_2019.xlsx')
make_X(X_2020).to_excel('X_2020.xlsx')
make_X(X_2021).to_excel('X_2021.xlsx')

y_2018.to_excel('y_2018.xlsx')
y_2019.to_excel('y_2019.xlsx')
y_2020.to_excel('y_2020.xlsx')
y_2021.to_excel('y_2021.xlsx')

C:\Users\Sunwoo\AppData\Local\Temp\ipykernel_2152\2023834807.py:16: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  X = pd.merge(X.reset_index(), info_drop, how ='left', on='사업자등록번호')
C:\Users\Sunwoo\AppData\Local\Temp\ipykernel_2152\2023834807.py:16: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  X = pd.merge(X.reset_index(), info_drop, how ='left', on='사업자등록번호')
C:\Users\Sunwoo\AppData\Local\Temp\ipykernel_2152\2023834807.py:16: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  X = pd.merge(X.reset_index(), info_drop, how ='left', on='사업자등록번호')
C:\Users\Sunwoo\AppData\Local\Temp\ipykernel_2152\2023834807.py:16: FutureWarning: merging between different levels is deprecated and will be removed in a futur